In [2]:
import pandas as pd
from sqlalchemy import create_engine
import mysql_conn

ModuleNotFoundError: No module named 'mysql_conn'

In [3]:
avacados = "avocado.csv"

avacado_df=pd.read_csv(avacados)

In [9]:
trim_avacado=avacado_df.drop(['Unnamed: 0','4046','4225','4770','Total Bags','Small Bags','Large Bags','XLarge Bags','year'],axis=1)
trim_avacado.head()

,Date,AveragePrice,Total Volume,type,region
0,2015-12-27,1.33,64236.62,conventional,Albany
1,2015-12-20,1.35,54876.98,conventional,Albany
2,2015-12-13,0.93,118220.22,conventional,Albany
3,2015-12-06,1.08,78992.15,conventional,Albany
4,2015-11-29,1.28,51039.60,conventional,Albany


In [ ]:
# renaming columns sample  new_county=county.rename(columns={'ID':'id',
                                       #'County Name (Licensee)':'name',
                                       #'County ID Code':'county_id',
                                       #'License Count':'license_count'
                                       #   })

In [ ]:
# set index